# Import Dependencies. Make sure to run in Python 3

In [7]:
import gseapy as gp
import pandas as pd
import numpy as np

# GSEA Prerank Function Block

In [8]:
def filter_cuff(cuff_file,fdr,fc,fpkm,flip):
    
    cuff_df = pd.read_table(cuff_file)
    print(len(cuff_df),'pre_filt')
    cuff_df = cuff_df[cuff_df.q_value < fdr]
    print(len(cuff_df),'post_fdr')
    cuff_df = cuff_df[cuff_df['log2(fold_change)'].abs() > fc]
    print(len(cuff_df),'post_fc')
    cuff_df = cuff_df[(cuff_df.value_1 >= fpkm) | (cuff_df.value_2 >= fpkm)]
    print(len(cuff_df),'post_fpkm\n')
    
    #Flip the log2(fold_change) sign if KD experiment because + log2FC should mean protein promotes expression
    if flip == 'yes':
        cuff_df['log2(fold_change)'] = cuff_df['log2(fold_change)'] * -1
    
    #now add ranking metrics
    cuff_df['neglogp'] = cuff_df.apply(lambda row: -1*np.log10(row['p_value'])*np.sign(row['log2(fold_change)']),axis=1)
    cuff_df['combo'] = cuff_df.apply(lambda row: -1*np.log10(row['p_value'])*row['log2(fold_change)'],axis=1)
    
    cuff_df.sort_values(by='log2(fold_change)',inplace=True,ascending=False)
    
    return(cuff_df)

fdr = 1
fc = 0
fpkm = 5

flip = 'no'

akemt_cuff = filter_cuff('../cuffdiff_runs/gencode_v24_comp/HTEMT_cuffdiff/gene_exp.diff',fdr,fc,fpkm,flip) 


60259 pre_filt
19195 post_fdr
19195 post_fc
10511 post_fpkm



In [10]:
#Need a rank file like this - needs to be in the form of dataframe
# 	0	1
# 0	ERG	6.687611
# 1	NRAP	5.179271
# 2	PSMB5	4.577776
# 3	PPFIBP2	4.568725
# 4	CNOT2	4.540159

def prep_rank_df_cuff(filt_df):
    print(len(filt_df),'total_number_of_genes_considered')
    #Break into the three rank statistics
    rank_stats = ['log2(fold_change)','neglogp','combo']
    rank_df_list = []
    for i in range(len(rank_stats)):
        select_cols = ['gene',rank_stats[i]]
        select_df = filt_df[select_cols]
        select_df = select_df.sort_values(by=rank_stats[i],ascending=False)
        #now need to drop duplicates
        select_df.drop_duplicates(subset='gene',keep='first',inplace=True)
        select_df.columns = 0,1
        print(len(select_df),'genes_after_dup_drop')
        #now need to drop infinite values
        select_df = select_df.replace([np.inf, -np.inf], np.nan).dropna()
        print(len(select_df),'genes_after_inf_drop')
        rank_df_list.append(select_df)
    return(rank_df_list)

#Ok now prep them based on rank statistics

#Genes

akemt_cuff_rnk = prep_rank_df_cuff(akemt_cuff)

10511 total_number_of_genes_considered
10509 genes_after_dup_drop
10502 genes_after_inf_drop
10509 genes_after_dup_drop
10509 genes_after_inf_drop
10509 genes_after_dup_drop
10502 genes_after_inf_drop


In [12]:
akemt_cuff_rnk[0].head()

,0,1
9832,PIEZO2,12.6527
10003,GDF6,11.7721
7258,DGKB,11.6049
13707,GLIS1,11.0176
13267,SNTB1,10.2593


In [13]:
def run_gsea_prerank_cuff(rank_df_list,gene_set,name,set_name):
    ranks = ['log2FC','neglogp','combo']
    output_list = []
    for i in range(len(rank_df_list)):
        output = gp.prerank(rnk=rank_df_list[i],gene_sets=gene_set,
                            processes=4, permutation_num=100, min_size = 5,
                            outdir='../gene_ontology/gsea_prerank/'+name+'/'+set_name+'.'+ranks[i],no_plot=False,format='png')
        rank_df_list[i].to_csv('../gene_ontology/gsea_prerank/'+name+'/'+set_name+'.'+ranks[i]+'.rnk',index=False,header=False,sep='\t')
        output_list.append(output)
    return(output_list)

# GSEA Prerank Running Block

In [16]:
gene_set = '/media/sam/Data2/annotations/msigdb_annotations/h.all.v6.2.symbols.gmt'
set_name = 'hallmarks'

#EMT
rank_df_list = akemt_cuff_rnk
name = 'htemt_cuff'
htemt_hallmarks = run_gsea_prerank_cuff(rank_df_list,gene_set,name,set_name)

In [17]:
!head /media/sam/Data2/annotations/msigdb_annotations/db_annotations/h.all.v6.2.symbols.gmt

HALLMARK_TNFA_SIGNALING_VIA_NFKB	http://www.broadinstitute.org/gsea/msigdb/cards/HALLMARK_TNFA_SIGNALING_VIA_NFKB	JUNB	CXCL2	ATF3	NFKBIA	TNFAIP3	PTGS2	CXCL1	IER3	CD83	CCL20	CXCL3	MAFF	NFKB2	TNFAIP2	HBEGF	KLF6	BIRC3	PLAUR	ZFP36	ICAM1	JUN	EGR3	IL1B	BCL2A1	PPP1R15A	ZC3H12A	SOD2	NR4A2	IL1A	RELB	TRAF1	BTG2	DUSP1	MAP3K8	ETS2	F3	SDC4	EGR1	IL6	TNF	KDM6B	NFKB1	LIF	PTX3	FOSL1	NR4A1	JAG1	CCL4	GCH1	CCL2	RCAN1	DUSP2	EHD1	IER2	REL	CFLAR	RIPK2	NFKBIE	NR4A3	PHLDA1	IER5	TNFSF9	GEM	GADD45A	CXCL10	PLK2	BHLHE40	EGR2	SOCS3	SLC2A6	PTGER4	DUSP5	SERPINB2	NFIL3	SERPINE1	TRIB1	TIPARP	RELA	BIRC2	CXCL6	LITAF	TNFAIP6	CD44	INHBA	PLAU	MYC	TNFRSF9	SGK1	TNIP1	NAMPT	FOSL2	PNRC1	ID2	CD69	IL7R	EFNA1	PHLDA2	PFKFB3	CCL5	YRDC	IFNGR2	SQSTM1	BTG3	GADD45B	KYNU	G0S2	BTG1	MCL1	VEGFA	MAP2K3	CDKN1A	CYR61	TANK	IFIT2	IL18	TUBB2A	IRF1	FOS	OLR1	RHOB	AREG	NINJ1	ZBTB10	PPAP2B	KLF4	CXCL11	SAT1	CSF1	GPR183	PMEPA1	PTPRE	TLR2	CXCR7	KLF10	MARCKS	LAMB3	CEBPB	TRIP10	F2RL1	KLF9	LDLR	TGIF1	RNF19B	DRAM1	B4GALT1	DNAJB4	CSF2	PDE4B	SNN	PLEK	STAT5A	D